In [93]:
import math
import numpy as np
import pandas as pd

def main():
    filename = 'titanic-homework.csv'
    data = pd.read_csv(filename)
    del data['PassengerId']
    del data['Name']
    
    decision_nodes = {}  # Liście drzewa decyzyjnego
    
    set_entropy = get_set_entropy(data)
    
    # Tworzenie wierzchołków
    for column in data:
        if column == 'Survived':
            continue
        g, l, e, q = get_decision_node_data(column, data)
        decision_nodes[column] = Node(column, g, l, e, q)
        
    tree = ''  # Korzeń
    
    # Ustalanie korzenia
    max_ratio = 0
    for n in decision_nodes:
        if decision_nodes[n].gain_ratio > max_ratio:
            max_ratio = decision_nodes[n].gain_ratio
            tree = decision_nodes[n]
            
    get_next_decision_node(tree, decision_nodes, data)
    
    print_tree(tree)
    
        
def get_decision_node_data(name, data):
    col = data[name]
    sums = {}
    trues = {}
    entropies = {}
    decisions = {}
    
    # Obliczanie ilości pasażerów z daną etykietą i ilości uratowanych pasażerów z daną etykietą
    for i in range(0, len(col)):
        if name == 'Age':
            if col[i] <= 20:
                label = 'young'
            elif col[i] <= 40:
                label = 'middle'
            else:
                label = 'old'
            if label in sums.keys():
                sums[label] += 1
            else:
                sums[label] = 1
            if data['Survived'][i] == 1:
                if label in trues.keys():
                    trues[label] += 1
                else:
                    trues[label] = 1
        else:
            if col[i] in sums.keys():
                sums[col[i]] += 1
            else:
                sums[col[i]] = 1
                trues[col[i]] = 0

            if data['Survived'][i] == 1:
                trues[col[i]] += 1
            
    conditional = 0  # Entropia warunkowa
    intrinsic = 0  # Intrinsic information
    for key in sums:
        entropies[key] = 0
        falses = sums[key] - trues[key]
        if trues[key] != 0:
            entropies[key] -= (trues[key] / sums[key]) * math.log2(trues[key] / sums[key])
        if falses != 0:
            entropies[key] -= (falses / sums[key]) * math.log2(falses / sums[key])

        conditional += (sums[key] / len(col)) * entropies[key]
        intrinsic -= (sums[key] / len(col)) * math.log2(sums[key] / len(col))
                                                          
    gain = get_set_entropy(data) - conditional  # Information gain
    if intrinsic != 0:
        gain_ratio = gain / intrinsic  # Gain ratio
    else:
        gain_ratio = 0
    
    next_labels = []
    next_entropies = []
    for e in entropies:
        next_labels.append(e)
        next_entropies.append(entropies[e])
    
    return gain_ratio, next_labels, next_entropies, len(col)


# Następny wierzchołek
def get_next_decision_node(node, nodes, data):
    nodes.pop(node.name)
    node.next_nodes = []
    # Warunek końca
    if len(nodes) == 0:
        for l in range(0, len(node.next_labels)):
            v, q = get_value_and_quantity(node.name, node.next_labels[l], data)
            node.next_nodes.append(Node(v, None, None, None, q))
        return
        
    # Znalezienie wierzchołka o największej entropii
    max_entropy_label = 0
    max_entr = 0
    for e in range(0, len(node.next_entropies)):
        if node.next_entropies[e] > max_entr:
            max_entr = node.next_entropies[e]
            max_entropy_label = e
           
    # Przypisywanie etykiety i ilości do liści
    for l in range(0, len(node.next_labels)):
        if l != max_entropy_label:
            v, q = get_value_and_quantity(node.name, node.next_labels[l], data)
            node.next_nodes.append(Node(v, None, None, None, q))
    
    # Usuwanie niepotrzebnych danych
    for l in node.next_labels:
        if l != node.next_labels[max_entropy_label]:
            data.drop(data[data[node.name] == l].index, inplace = True)
            data.reset_index(drop=True, inplace=True)
            
    # znalezienie wierzchołka o obecnie największym information ratio
    max_ratio = 0
    max_ratio_node = 0
    next_node = None
    for n in nodes:
        r, l, e, q = get_decision_node_data(nodes[n].name, data)
        if r >= max_ratio:
            max_ratio = r
            max_ratio_node = n
    nodes[max_ratio_node].class_quantity = q
    next_node = Node(nodes[max_ratio_node].name, nodes[max_ratio_node].gain_ratio, nodes[max_ratio_node].next_labels, nodes[max_ratio_node].next_entropies, nodes[max_ratio_node].class_quantity)
            
    node.next_nodes.append(next_node)
    get_next_decision_node(next_node, nodes, data)
    return
             
    
# Ilość i najczęstszy przypadek dla danego atrybutu
def get_value_and_quantity(column, attr, data):
    col = data[column]
    q = 0
    t = 0
    v = 'Nie'
    
    for i in range(0, len(col)):
        if col[i] == attr:
            q += 1
            if data['Survived'][i] == 1:
                t += 1
            
    if t > q/2:
        v = 'Tak'
        
    return v, q


# Entropia całości
def get_set_entropy(data):
    s = 0
    for v in data['Survived']:
        if v == 1:
            s += 1
    f = len(data['Survived']) - s
    if f != 0:
        return -(s/len(data['Survived'])) * math.log2(s/len(data['Survived'])) - (f/len(data['Survived'])) * math.log2(f/len(data['Survived']))
    else:
        return 0

# 
def print_tree(node):
    if node.name in ('Tak', 'Nie'):
        return
    
    print(node.name, node.class_quantity)
    for i in range(0, len(node.next_labels)):
        print(node.next_labels[i], '->', node.next_nodes[i].name)
    
    for n in node.next_nodes:
        print_tree(n)
        

# Liść
class Node:
    name = ''
    gain_ratio = 0
    next_labels = []
    next_entropies = []
    class_quantity = 0
    next_nodes = None
        
    def __init__(self, name, gain_ratio, next_labels, next_entropies, class_quantity):
        self.name = name
        self.gain_ratio = gain_ratio
        self.next_labels = next_labels
        self.next_entropies = next_entropies
        self.class_quantity = class_quantity
    
if __name__ == '__main__':
    main()

Sex 100
male -> Nie
female -> SibSp
SibSp 40
1 -> Tak
0 -> Tak
3 -> Tak
4 -> Nie
2 -> Nie
5 -> Pclass
Pclass 15
3 -> Nie
1 -> Tak
2 -> Parch
Parch 4
0 -> Tak
1 -> Nie
2 -> Nie
5 -> Nie
3 -> Age
Age 0
middle -> Nie
old -> Nie
young -> Nie
